In [3]:
from utils import load_embedding, parse_tup, cos_sim, cos_dist
from sklearn.decomposition import PCA
from tqdm import tqdm
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import pandas as pd

In [4]:
TIME_FRAME = "biweekly"
subreddits, vectors  = load_embedding("/h/224/cameron/Political-Subreddit-Embedding/trained_embeddings/adapted_vecs.txt")
embedding = pd.concat([subreddits, vectors], axis=1).set_index("subreddit")
del subreddits, vectors
embedding

,1,2,3,4,5,6,7,8,9,10,...,141,142,143,144,145,146,147,148,149,150
subreddit,,,,,,,,,,,,,,,,,,,,,
SquaredCircle,-0.006950,-0.085116,-0.018554,-0.172390,0.018786,-0.127979,-0.005660,-0.129364,0.063671,0.059154,...,0.010085,0.041204,-0.058507,-0.064563,-0.005662,0.093066,0.091820,-0.099796,0.150331,-0.085893
AskReddit,-0.012029,-0.067604,-0.003573,-0.065879,0.024166,-0.060270,-0.109945,-0.071275,0.134388,-0.074362,...,-0.007804,-0.031531,-0.198097,0.067012,-0.101151,0.092231,0.088949,-0.041761,-0.112736,-0.053113
politics,-0.133536,-0.101922,-0.034117,-0.093723,0.030623,-0.068788,0.056715,-0.093384,0.139595,-0.084614,...,-0.025986,0.084037,-0.190394,-0.032979,-0.095009,0.021711,0.044770,-0.121603,-0.009190,0.020746
memes,-0.097242,0.043635,-0.092566,0.007617,-0.065151,-0.050654,0.043962,-0.055058,-0.063219,-0.073888,...,0.043389,-0.016802,-0.127256,0.138457,-0.101161,-0.050914,0.060248,-0.158746,-0.082112,-0.052449
teenagers,-0.089968,0.044256,-0.051055,0.045911,-0.051108,-0.069145,-0.003921,-0.052291,0.030269,-0.126074,...,0.017685,-0.008737,-0.175991,0.069359,-0.126235,-0.003090,0.000458,-0.100655,-0.120615,-0.107238
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NRV,0.085428,-0.043005,-0.019434,-0.135469,-0.008682,-0.037592,-0.030862,-0.028504,0.094053,-0.156332,...,-0.021320,0.022850,-0.171799,0.087259,-0.047655,0.075187,0.064219,-0.071066,-0.033605,-0.086041
UTT,-0.007385,-0.050983,0.006738,-0.142325,-0.005179,-0.055992,-0.016312,-0.071271,0.076586,-0.175807,...,-0.054637,0.018867,-0.178137,0.053923,-0.104482,0.071664,0.036262,-0.064455,-0.041796,-0.062654
MMU,0.014398,-0.081064,0.013574,-0.127131,-0.018591,-0.063662,-0.003867,-0.051093,0.098597,-0.156805,...,-0.026334,0.002479,-0.184596,0.063692,-0.082224,0.112338,0.041958,-0.093367,-0.021291,-0.047175


In [ ]:
tqdm.pandas()
def find_closest(transformation,subreddit="politics",vec=None,exclude=[]):
    # Get subreddit
    assert subreddit or vec, "Must supply a subreddit name (that's in the data frame), or the vector representation of a subreddit"
    assert subreddit in embedding.index.values, "{} is not in the dataset".format(subreddit)
    sub_vec = embedding.loc[subreddit].to_numpy().flatten() if subreddit else vec
    transformed = sub_vec + transformation
    all_other_vecs = embedding[~embedding.index.isin(exclude+[subreddit])]
    similarity = np.apply_along_axis(lambda x : cos_sim(transformed,x), 1, all_other_vecs)
    winner_ind = similarity.argmax()
    return all_other_vecs.index.values[winner_ind], similarity[winner_ind]
    
def get_transformation(sub_a,sub_ap):
    """
        sub_a is to sub_ap
        sub_ap - sub_a
        returns transformation from a to ap
    """
    assert sub_a in embedding.index.values, "{} is not in the dataset".format(sub_a)
    assert sub_ap in embedding.index.values, "{} is not in the dataset".format(sub_ap)
    vec_a = embedding.loc[sub_a].to_numpy().flatten()
    vec_ap = embedding.loc[sub_ap].to_numpy().flatten()
    return vec_ap - vec_a

def get_possible_fits(a,ap,k=10):
    trans = get_transformation(a,ap)
    possible_fits = embedding.progress_apply(lambda x : find_closest(trans,vec=x,exclude=[a,ap]))
    possible_fits_df = pd.DataFrame(possible_fits, columns=['subreddit_to', 'transformation_similarity'])
    possible_fits_df["subreddit_from"] = embedding.index
    return possible_fits_df

a = "socialism"
ap = "Libertarian"
b = "SandersForPresident"
bp = "torontoraptors"
p = get_possible_fits(a,ap)

 61%|██████    | 91/150 [02:33<01:31,  1.55s/it]

In [10]:
possible_fits

NameError: name 'possible_fits' is not defined

In [13]:
def analogy(sub_a,sub_ap,sub_b):
    """
        sub_a is to sub_ap, as sub_b is to ??
        sub_ap + sub_b - sub_a
        returns the subreddit that best answers the analogy
    """
    assert sub_a in embedding.index.values, "{} is not in the dataset".format(sub_a)
    assert sub_ap in embedding.index.values, "{} is not in the dataset".format(sub_ap)
    assert sub_b in embedding.index.values, "{} is not in the dataset".format(sub_b)
    vec_a = embedding.loc[sub_a].to_numpy().flatten()
    vec_ap = embedding.loc[sub_ap].to_numpy().flatten()
    vec_b = embedding.loc[sub_b].to_numpy().flatten()
    vec_bp = vec_ap + (vec_b - vec_a)
    all_other_vecs = embedding[~embedding.index.isin([sub_a,sub_ap,sub_b])]
    similarity = np.apply_along_axis(lambda x : cos_sim(x,vec_bp), 1, all_other_vecs)
    winner_ind = similarity.argmax()
    return all_other_vecs.index.values[winner_ind]

a = "socialism"
ap = "Libertarian"
b = "SandersForPresident"
print("r/{} is to r/{} as r/{} is to r/{}".format(a,ap,b,analogy(a,ap,b)))

r/socialism is to r/Libertarian as r/SandersForPresident is to r/hottiesfortrump


In [4]:
from itertools import combinations

def predict_analogies(analogies):
    tuples = [tuple(x) for x in analogies.to_numpy()]
    pairs = list(combinations(tuples,2))
    pred = []
    actual = []
    for (a,ap),(b,bp) in tqdm(pairs):
        try: 
            p = analogy(a.strip(),ap.strip(),b.strip())
            pred.append(p)
            actual.append(bp)
        except:
            pass
    return np.array(pred),np.array(actual), pairs

analogies = pd.read_csv("/h/224/cameron/Political-Subreddit-Embedding/external_datasets/validation/uni_to_city.csv",nrows=50)
pred, actual, tuples = predict_analogies(analogies)
accuracy = (pred == actual).mean() * 100
num_correct = (pred == actual).sum()
print("Out of {} analogies, {} were predicted correctly ({}%)".format(len(pred),num_correct,accuracy))
for p,act,((a,ap),(b,_)) in zip(pred,actual,tuples):
    if p==act: print("{} is to {} as {} is to {} (predicted as: {})".format(a,ap,b,act,p))


100%|██████████| 1225/1225 [02:26<00:00,  8.36it/s]

Out of 990 analogies, 175 were predicted correctly (17.67676767676768%)
Algonquin_College is to ottawa as aquinas is to canberra (predicted as: canberra)
Algonquin_College is to ottawa as Baruch is to Binghamton (predicted as: Binghamton)
Algonquin_College is to ottawa as BGSU is to burnaby (predicted as: burnaby)
Algonquin_College is to ottawa as brandeis is to fortlauderdale (predicted as: fortlauderdale)
Algonquin_College is to ottawa as butleruniversity is to SLO (predicted as: SLO)
Algonquin_College is to ottawa as calarts is to ChicoCA (predicted as: ChicoCA)
Algonquin_College is to ottawa as CSUC is to longbeach (predicted as: longbeach)
Algonquin_College is to ottawa as CSULB is to Sacramento (predicted as: Sacramento)
Algonquin_College is to ottawa as csumb is to fresno (predicted as: fresno)
Algonquin_College is to ottawa as CapU is to canberra (predicted as: canberra)
amarillocollege is to amarillo as amherstcollege is to Muncie (predicted as: Muncie)
amarillocollege is to a